<a href="https://colab.research.google.com/github/kyowoii23/system_quality_project/blob/main/notebooks/system_quality_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 데이터 불러오기

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
train_err = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/인공지능 머신러닝_모델링/data/system_quality_project/train_err_data.csv')
train_problem = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/인공지능 머신러닝_모델링/data/system_quality_project/train_problem_data.csv')
train_quality = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/인공지능 머신러닝_모델링/data/system_quality_project/train_quality_data.csv')
sample = pd.DataFrame(pd.read_csv('/content/drive/MyDrive/Colab Notebooks/인공지능 머신러닝_모델링/data/system_quality_project/sample_submission.csv'))

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (10,11,12) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
sample

,user_id,problem
0,30000,0
1,30001,0
2,30002,0
3,30003,0
4,30004,0
...,...,...
14994,44994,0
14995,44995,0
14996,44996,0
14997,44997,0


## 1_1. 컬럼별 설명
- train_err_data
  - `user_id` : 사용자 ID
  - `time` : 로그발생시간
  - `model_nm` : 모델명
  - `fwver` : 펌웨어버전
  - `errtype` : 에러타입
  - `errcode` : 에러코드

- train_problem_data
  - `user_id` : 사용자 ID
  - `time` : 불만접수시간

- train_quality_data
  - `time` : 로그발생시간
  - `user_id` : 사용자 ID
  - `fwver` : 펌웨어버전
  - `quality_0 ~ 12` : 퀄리티_0 ~ 12

In [5]:
print(train_err.shape)
print(train_problem.shape)
print(train_quality.shape)

(16554663, 6)
(5429, 2)
(828624, 16)


In [6]:
print(train_err.info())
print(train_problem.info())
print(train_quality.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16554663 entries, 0 to 16554662
Data columns (total 6 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   user_id   int64 
 1   time      int64 
 2   model_nm  object
 3   fwver     object
 4   errtype   int64 
 5   errcode   object
dtypes: int64(3), object(3)
memory usage: 757.8+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5429 entries, 0 to 5428
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   user_id  5429 non-null   int64
 1   time     5429 non-null   int64
dtypes: int64(2)
memory usage: 85.0 KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 828624 entries, 0 to 828623
Data columns (total 16 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   time        828624 non-null  int64  
 1   user_id     828624 non-null  int64  
 2   fwver       788544 non-null  object 
 3   quality_0   684192 non-null  floa

# 2. Data Preprocessing

In [7]:
print('---train_err---')
print(train_err.isnull().sum())

print('---train_problem---')
print(train_problem.isnull().sum())

print('---train_quality---')
print(train_quality.isnull().sum())

---train_err---
user_id     0
time        0
model_nm    0
fwver       0
errtype     0
errcode     1
dtype: int64
---train_problem---
user_id    0
time       0
dtype: int64
---train_quality---
time               0
user_id            0
fwver          40080
quality_0     144432
quality_1          0
quality_2      40113
quality_3          0
quality_4          0
quality_5         20
quality_6          0
quality_7          0
quality_8          0
quality_9          0
quality_10         0
quality_11         0
quality_12         0
dtype: int64


In [8]:
train_err.head(10)

,user_id,time,model_nm,fwver,errtype,errcode
0,10000,20201101025616,model_3,05.15.2138,15,1
1,10000,20201101030309,model_3,05.15.2138,12,1
2,10000,20201101030309,model_3,05.15.2138,11,1
3,10000,20201101050514,model_3,05.15.2138,16,1
4,10000,20201101050515,model_3,05.15.2138,4,0
5,10000,20201101050516,model_3,05.15.2138,26,1
6,10000,20201101050522,model_3,05.15.2138,4,0
7,10000,20201101060903,model_3,05.15.2138,15,1
8,10000,20201101174754,model_3,05.15.2138,16,1
9,10000,20201101174756,model_3,05.15.2138,4,0


In [9]:
train_problem.head(10)

,user_id,time
0,19224,20201102200000
1,23664,20201116140000
2,15166,20201114130000
3,12590,20201108210000
4,15932,20201103210000
5,16852,20201119150000
6,23427,20201121110000
7,13507,20201111160000
8,11274,20201118120000
9,20610,20201127230000


In [10]:
train_quality.head(30)

,time,user_id,fwver,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
1,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
2,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
3,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
4,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
5,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
6,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,4,0,0,0,0,4,0,0
7,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
8,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
9,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0


In [11]:
train_problem['time'].agg(['min', 'max'])

min    20201101000000
max    20201130230000
Name: time, dtype: int64

In [12]:
train_quality['time'].agg(['min', 'max'])

min    20201031235000
max    20201130234000
Name: time, dtype: int64

In [13]:
train_err['time'].agg(['min', 'max'])

min    20201031235959
max    20201202185152
Name: time, dtype: int64

In [14]:
# model_nm -> model_0 ~ 8
print('model_nm : \n',np.sort(train_err.model_nm.unique()))
print()

# fwver의 개수는 37
print('fwver : \n',train_err.fwver.nunique())
print()

# errtype의 종류
print('errtype : \n',np.sort(train_err.errtype.unique()))
print()

# train_err.errcode.unique() # dtype = object
print('errcode : \n',train_err.errcode.nunique()) # 2805개

model_nm : 
 ['model_0' 'model_1' 'model_2' 'model_3' 'model_4' 'model_5' 'model_6'
 'model_7' 'model_8']

fwver : 
 37

errtype : 
 [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 30 31 32 33 34 35 36 37 38 39 40 41 42]

errcode : 
 2805


quality는 사용자의 시스템 작동 중 문제가 발생하면 측정 가능한 지표들로 해당 시점으로부터 2시간동안 10분 간격으로 찍힌 데이터입니다.
1. quality0 ~ 12까지의 숫자들의 의미는 무엇인가? 발생횟수?
2. 왜 quality의 숫자마다 데이터타입이 다를까?
3. 각 row당 0~12까지 찍힌 개수에 따라 등급을 부여한다면?
4. quality 결측값?

In [15]:
train_user_id_max = 24999
train_user_id_min = 10000
train_user_number = 15000

In [16]:
train_err['user_id'].nunique()

15000

In [17]:
id_error = train_err[['user_id', 'errtype']].values
id_error

array([[10000,    15],
       [10000,    12],
       [10000,    11],
       ...,
       [24999,     4],
       [24999,     4],
       [24999,    15]])

In [18]:
id_error.shape

(16554663, 2)

In [19]:
errors = np.zeros((train_user_number, 42))
errors

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [20]:
# for person_id, err in id_error:
#   errors[person_id - train_user_id_min, err - 1] += 1

In [21]:
errors

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [22]:
errors.shape

(15000, 42)

train_quality
1. 두시간동안 10분단위로 찍히는 로그라고 했는데 time이 모두 같음, 12개의 로그는 맞음
2. time이 다 같지만 quality번호당 적혀있는 숫자가 다른 경우가 있음
3. 

In [25]:
train_quality.head(50)

,time,user_id,fwver,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
1,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
2,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
3,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
4,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
5,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
6,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,4,0,0,0,0,4,0,0
7,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
8,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
9,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0


In [30]:
train_quality['fwver'].value_counts()

05.15.2138    163236
04.22.1750    142032
04.33.1261    131340
04.16.3553    111996
03.11.1167    104148
04.33.1185     66024
04.22.1778     56472
04.22.1684      3420
09.17.1431      3384
04.16.3571      2232
04.82.1684      1044
04.33.1149       804
04.22.1666       636
05.66.3237       552
04.82.1778       216
03.11.1149       204
04.73.2237       192
04.33.1125       144
05.66.3571        96
05.15.2114        96
05.15.2120        84
04.22.1442        84
05.15.2122        60
04.22.1656        12
04.16.3439        12
04.73.2571        12
04.16.3345        12
Name: fwver, dtype: int64

In [31]:
train_quality.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 828624 entries, 0 to 828623
Data columns (total 16 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   time        828624 non-null  int64  
 1   user_id     828624 non-null  int64  
 2   fwver       788544 non-null  object 
 3   quality_0   684192 non-null  float64
 4   quality_1   828624 non-null  int64  
 5   quality_2   788511 non-null  float64
 6   quality_3   828624 non-null  int64  
 7   quality_4   828624 non-null  int64  
 8   quality_5   828604 non-null  object 
 9   quality_6   828624 non-null  int64  
 10  quality_7   828624 non-null  object 
 11  quality_8   828624 non-null  object 
 12  quality_9   828624 non-null  object 
 13  quality_10  828624 non-null  object 
 14  quality_11  828624 non-null  int64  
 15  quality_12  828624 non-null  int64  
dtypes: float64(2), int64(8), object(6)
memory usage: 101.2+ MB


In [32]:
train_quality.isnull().sum()

time               0
user_id            0
fwver          40080
quality_0     144432
quality_1          0
quality_2      40113
quality_3          0
quality_4          0
quality_5         20
quality_6          0
quality_7          0
quality_8          0
quality_9          0
quality_10         0
quality_11         0
quality_12         0
dtype: int64

In [45]:
train_quality['quality_10'].unique()

array(['4', '8', '0', ..., '108,441', '9,558', '1,540'], dtype=object)

In [36]:
for i in range(0,12):
  print(f'-------quality{i}--------')
  print(train_quality[f'quality_{i}'].value_counts())
  print()

-------quality0--------
 0.0       542790
-1.0       130828
 1.0         2097
 2.0         1252
 3.0          518
            ...  
 1113.0         1
 1177.0         1
 1186.0         1
 946.0          1
 2351.0         1
Name: quality_0, Length: 753, dtype: int64

-------quality1--------
 0      670270
-1      153649
 1        2567
 2        1140
 3         391
 4         202
 5         134
 6          71
 7          50
 8          35
 9          26
 10         24
 11         12
 12          7
 13          6
 14          5
 21          4
 20          4
 170         4
 18          4
 17          3
 15          3
 16          3
 169         2
 23          2
 35          1
 117         1
 171         1
 19          1
 88          1
 40          1
Name: quality_1, dtype: int64

-------quality2--------
 0.0        632469
-1.0        144392
 1.0          2937
 2.0          1073
 3.0           580
             ...  
 872.0           1
 11259.0         1
 373.0           1
 8278.0          1
